In [36]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

import warnings
warnings.filterwarnings('ignore')


In [37]:
trainingData = pd.read_csv('./data/train.csv')
testData = pd.read_csv('./data/test.csv')


In [48]:
from collections import Counter
import re


def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name)
    if title_search:
        return title_search.group(1)
    return ""


# 敬称を新たな列として追加
trainingData['Title'] = trainingData['Name'].apply(get_title)

testData['Title'] = testData['Name'].apply(get_title)


# trainingDataのAgeの平均値を算出
age_mean = trainingData['Age'].mean()

titles = list(trainingData['Title'])
# title_namesのリストの中にある敬称を持つ人の平均年齢を算出し、dictに格納　
title_ages = {}
for title in titles:
    title_ages[title] = trainingData[trainingData['Title']
                                     == title]['Age'].mean()

# リスト2つからdictを作成

# 年齢が欠損しているレコードを、その人の敬称に対応する平均年齢で補完
trainingData.loc[trainingData['Age'].isnull(), 'Age'] = trainingData[trainingData['Age'].isnull()].apply(
    lambda row: title_ages[row['Title']], axis=1)

testData.loc[testData['Age'].isnull(), 'Age'] = testData[testData['Age'].isnull()].apply(
    lambda row: title_ages[row['Title']], axis=1)

# testDataのAgeが欠損している値を、age_meanで補完
testData.loc[testData['Age'].isnull(), 'Age'] = age_mean


In [50]:
trainingData.drop('Cabin', axis=1, inplace=True)
trainingData['Embarked'].fillna('S', inplace=True)
if trainingData.isnull().sum() > 0:
    print('nullがあります')

testData.drop('Cabin', axis=1, inplace=True)
testData['Embarked'].fillna('S', inplace=True)
testData.isnull().sum()


KeyError: "['Cabin'] not found in axis"

In [51]:
trainingData.replace({'Sex': {'male': 0, 'female': 1}}, inplace=True)
testData.replace({'Sex': {'male': 0, 'female': 1}}, inplace=True)

embarked_ohe = trainingData['Embarked']
embarked_ohe = pd.get_dummies(embarked_ohe)
trainingData = pd.concat([trainingData, embarked_ohe], axis=1)
trainingData.drop('Embarked', axis=1, inplace=True)

testData = pd.concat([testData, pd.get_dummies(testData['Embarked'])], axis=1)
testData.drop('Embarked', axis=1, inplace=True)
print(trainingData.head())
testData.head()


   PassengerId  Perished  Pclass  \
0            1         1       3   
1            2         0       1   
2            3         0       3   
3            4         0       1   
4            5         1       3   

                                                Name  Sex   Age  SibSp  Parch  \
0                            Braund, Mr. Owen Harris    0  22.0      1      0   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...    1  38.0      1      0   
2                             Heikkinen, Miss. Laina    1  26.0      0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)    1  35.0      1      0   
4                           Allen, Mr. William Henry    0  35.0      0      0   

             Ticket     Fare Title  C  Q  S  
0         A/5 21171   7.2500    Mr  0  0  1  
1          PC 17599  71.2833   Mrs  1  0  0  
2  STON/O2. 3101282   7.9250  Miss  0  0  1  
3            113803  53.1000   Mrs  0  0  1  
4            373450   8.0500    Mr  0  0  1  


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Title,C,Q,S
0,892,3,"Kelly, Mr. James",0,34.5,0,0,330911,7.8292,Mr,0,1,0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",1,47.0,1,0,363272,7.0000,Mrs,0,0,1
2,894,2,"Myles, Mr. Thomas Francis",0,62.0,0,0,240276,9.6875,Mr,0,1,0
3,895,3,"Wirz, Mr. Albert",0,27.0,0,0,315154,8.6625,Mr,0,0,1
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",1,22.0,1,1,3101298,12.2875,Mrs,0,0,1


In [58]:
trainingData.drop(
    ['Name', 'Ticket', 'Fare', 'Title'], axis=1, inplace=True)
testData.drop(['Name', 'Ticket', 'Fare', 'Title'], axis=1, inplace=True)
X = trainingData.iloc[:, 2:].values
y = trainingData.iloc[:, 1].values


KeyError: "['Name' 'Ticket' 'Fare' 'Title'] not found in axis"

In [53]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV


param_grid = {'max_depth': [3, 5, 6, 7, 8, 9],
              'min_samples_leaf': [1, 2, 3, 4, 5]}


# param_grid = {'max_depth': [3, 5, 7],
            #   'min_samples_leaf': [1, 2, 4]}

X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.3, random_state=42)


rfc_gs = GridSearchCV(RandomForestClassifier(
    n_estimators=100, n_jobs=-1, random_state=42), param_grid, cv=5)
rfc_gs.fit(X, y)

print('Best Parameters: {}'.format(rfc_gs.best_params_))
print('CV Score: {}'.format(round(rfc_gs.best_score_, 3)))


Best Parameters: {'max_depth': 6, 'min_samples_leaf': 1}
CV Score: 0.828


In [54]:
from sklearn.neural_network import MLPClassifier


mlpc = MLPClassifier(hidden_layer_sizes=(100, 100, 10), random_state=0)
mlpc.fit(X_train, y_train)

print('Multilayer Perceptron \n')
print('Train Score: {}'.format(round(mlpc.score(X_train, y_train), 3)))
print(' Test Score: {}'.format(round(mlpc.score(X_valid, y_valid), 3)))


Multilayer Perceptron 

Train Score: 0.835
 Test Score: 0.817


In [55]:
lr = LogisticRegression(random_state=42)
lr.fit(X_train, y_train)

print('Logistic Regression \n')
print('Train Score: {}'.format(round(lr.score(X_train, y_train), 3)))
print(' Test Score: {}'.format(round(lr.score(X_valid, y_valid), 3)))


Logistic Regression 

Train Score: 0.795
 Test Score: 0.81


In [56]:
rfc = RandomForestClassifier(
    max_depth=10, min_samples_leaf=1, n_estimators=100, n_jobs=-1, random_state=42)
rfc.fit(X_train, y_train)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=10, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [63]:
testData.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,C,Q,S
0,892,3,0,34.5,0,0,0,1,0
1,893,3,1,47.0,1,0,0,0,1
2,894,2,0,62.0,0,0,0,1,0
3,895,3,0,27.0,0,0,0,0,1
4,896,3,1,22.0,1,1,0,0,1


In [65]:
testData.drop('PassengerId', axis=1, inplace=True)
rfc_pred = rfc.predict_proba(testData)
lr_pred = lr.predict_proba(testData)
mlpc_pred = mlpc.predict_proba(testData)

pred_proba = (rfc_pred + lr_pred + mlpc_pred) / 3
pred = pred_proba.argmax(axis=1)
pred


In [69]:
# 読み込むデータが格納されたディレクトリのパス，必要に応じて変更の必要あり
path = "./gender_submission.csv"
submission = pd.read_csv(path)
submission
submission['Perished'] = pred
# JupyterLabなどローカルで作業する場合
# パスは必要に応じて変更の必要あり
submission.to_csv('./my_Submittion.csv', index=False)
